## Hacer cambios en Modelos


In [ ]:
from django.db import models

# Create your models here.

class ProducModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField()
    color = models.TextField(null=True)
    short_description = models.TextField(null=True)

- python manage.py makemigrations
- python manage.py migrate

## Borrar un campo en los modelos

In [ ]:
from django.db import models

# Create your models here.

class ProducModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField()
    color = models.TextField(null=True)
    short_description = models.TextField(null=True)

- python manage.py makemigrations
- python manage.py migrate

## Borrar y comprimir una migración


pthon manage.py squashmigrations <APP_LABEL> <MIGRATION_NUMBER>

- pthon manage.py squashmigrations ecommerce 004
- pthon manage.py migrate


## Guardando data usando el shell de Django

- python manage.py shell

In [ ]:
>> from ecommerce.models import ProductModel
>> ProductModel.objects.create(title="Producto 1", price="199.99")


Queryset: Lista de todos los objetos almancenados para cierto modelo
    
    
queryset = ProductModel.objects.all()
qs = ProductModel.object.all()

queryset.filter(title__incontains="producto")

>> my_product = ProductModel.objects.get(id=18)
>> my_product.description
"Es una Prueba"
>> my_product.price
199.99
>> my_product.price = 299.99
>> my_product.save()

## Validaciones

ecommerce/validators.py

In [ ]:
from django.core.exceptions import ValidationError

LOCKED_WORDS = [
    "barato",
    "malo"
]

def validate_blocked_words(value):
    init_string = f"{value}",lower()
    unique_words = set(init_string.split())
    blocked_words = set(BLOCKED_WORDS)
    invalid_words = (unique_words & blocked_words)
    has_error = len(invalid_words) > 0

    if has_error:
        errors = []
        for invalid_word in invalid_words:
            msg = "{} es una palabra no permitida".format(invalid_word)
            errors.append(msg)
        raise ValidationError(errors)

    return value

## ecommerce/models.py

In [ ]:
from django.db import models
from .validators import validate_blocked_words

PUBLISH_STATE_CHOICES = [
    ("BR", "BORRADOR"),
    ("PU", "PUBLICADO"),
    ("PR", "PRIVADO"),
]


# Create your models here.

class ProducModel(models.Model):
    title = models.TextField()
    price = models.FloatField()
    description = models.TextField()
    color = models.TextField()

    def save(self, *args, **kwargs):
        validate_blocked_words (self.title) 
        super().save(*args, **kwargs)


class ProductModel(models.Model):
    state = models.CharField(max_length=2, choices=PUBLISH_STATE_CHOICES, default="BR")
    title =  models.TextField() 
    price =  models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs): 
        validate_blocked_words(self.title) 
        super().save(*args, **kwargs)

    def is_published(self): 
        return self.state == "PU"

## como agregar opciones avanzadas a los campos de nuestros Modelos


In [ ]:
from django.db import models
from .validators import validate_blocked_words


# Create your models here.

class ProducModel(models.Model):
    class ProducStateOptions(models.TextChoices):
        PUBLISHED = "PU", "PUBLICADO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"

    state = models.CharField(max_length=2, choices=ProducStateOptions.choices, default=ProducStateOptions.DRAFT)
    title =  models.TextField() 
    price =  models.FloatField()
    color =  models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs): 
        validate_blocked_words(self.title) 
        super().save(*args, **kwargs)

    def is_published(self): 
        return self.state == self.ProducStateOptions.PUBLISHED

## MOdelo abstracto de BASE

In [ ]:
python manage.py startapp base

In [ ]:
base/models.py

In [ ]:
from django.db import models
from django.utils import timezone

class BasePublishModel(models.Model):
    class PublishStateOptions (models.TextChoices):
        PUBLISHED = "PU", "PUBLICADO"
        DRAFT = "BR", "BORRADOR"
        PRIVATE = "PR", "PRIVADO"

    state = models.CharField(max_length=2, choices=PublishStateOptions.choices, default=PublishStateOptions.DRAFT)
    timestamp = models.DateTimeField(auto_now_add=True) 
    updated = models.DateTimeField(auto_now_add=True)
    publish_timestamp = models.DateTimeField(auto_now_add=False, auto_now=False, null=True)

    class Meta:
        abstract = True
        ordering = ["-updated", "-timestamp"]

    
    def save(self, *args, **kwargs): 
        if self.state_is_published and self.publish_timestamp is None:
            self.publish_timestamp = timezone.now()

        else: 
            self.publish_timestamp= None

        super().save(*args, **kwargs)

    @property
    def state_is_published(self):
        return self.state == self.PublishStateOptions.PUBLISHED

    def is_published(self):
        publish_timestamp = self.publish_timestamp
        return self.state_is_published and publish_timestamp < timezone.now()

In [ ]:
from django.db import models
from base.models import BasePublishModel
from .validators import validate_blocked_words


class ProducModel(BasePublishModel):
    title =  models.TextField() 
    price =  models.FloatField()
    color =  models.FloatField()
    description = models.TextField(null=True)

    def save(self, *args, **kwargs): 
        validate_blocked_words(self.title) 
        super().save(*args, **kwargs)


In [ ]:
config/settings.py

# Application definitions
INSTALLED_APPS = [
    "ecommerce.apps.EcommerceConfig",
    "pages.apps.PagesConfig",
    "base.apps.BaseConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]

In [ ]:
python manage.py makemigrations
python manage.py migrate

## Creacion a GRANEL

In [ ]:
products_data = [
    {"title": "Producto 1", "price": 19.99},
    {"title": "Producto 1", "price": 98.99},
    {"title": "Producto 1", "price": 84.99},
    {"title": "Producto 1", "price": 79.99},
    {"title": "Producto 1", "price": 9.99}
]

In [ ]:
products_data = []

for i in range(1, 11):
        new_data = {"title": "Producto {}".format(i), "price": i*100+99.99}
        products_data.append(new_data)

In [ ]:
new_objects = []

for product_data in products_data():
    print(product_data)
    new_objects.append(ProductModel(**product_data))

## SlugField y señales en Modelos

slug: se usa para los urls
title: escritorio con altura ajustable
slug: /escritorio-con-altura-ajustable
url: www.mywebsite.com/escritorio-con-altura-ajustable
url: www.mywebsite.com/1

Signals / Señales

- pre_save
- post_save
- pre_delete
- post_delete
- pre_init
- post_init
- pre_migrate
- post_migrate


In [ ]:
from django.db.models import signals
dir(signals)

In [ ]:
ecommerce/models.py

In [ ]:
from django.db import models
from django.db.models.signals import pre_save
from django.utils.text import slugify
from base.models import BasePublishModel
from .validators import validate_blocked_words


# Create your models here.

class ProducModel(BasePublishModel):
    title =  models.TextField() 
    price =  models.FloatField()
    color =  models.FloatField()
    description = models.TextField(null=True)

    def det_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs): 
        validate_blocked_words(self.title) 
        super().save(*args, **kwargs)

def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objetos.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"
            
pre_save.connect(slugify_pre_save, sender=ProducModel)

python manage.py makemigrations
python manage.py migrate

## Fixtures para cargar Data

In [ ]:
python manage.py dumpdata ecommerce --indent 4 --format json

In [ ]:
python manage.py sumpdata ecommerce --indent 4 --format json > ecommerce /fixtures/ecommerce/ProductModel.json

In [ ]:
from django.conf import settings
from django.db import models
from django.db.models.signals import pre_save
from django.utils.text import slugify
from base.models import BasePublishModel
from .validators import validate_blocked_words

User = settings.AUTH_USER_MODEL


# Create your models here.

class ProducModel(BasePublishModel):
    title =  models.TextField() 
    price =  models.FloatField()
    color =  models.FloatField()
    description = models.TextField(null=True)
    user = models.ForeignKey(User, null= True, on_delete=models.SET_NULL)

    def det_absolute_url(self):
        return f"/product/{self.slug}"

    def save(self, *args, **kwargs): 
        validate_blocked_words(self.title) 
        super().save(*args, **kwargs)

def slugify_pre_save(sender, instance, *args, **kwargs):
    if instance.slug is None or instance.slug == "":
        new_slug = slugify(instance.title)
        MyModel = instance.__class__
        qs = MyModel.objetos.filter(slug__startswith=new_slug).exclude(id=instance.id)
        if qs.count() == 0:
            instance.slug = new_slug
        else:
            instance.slug = f"{new_slug}-{qs.count()}"
            
pre_save.connect(slugify_pre_save, sender=ProducModel)

- python manage.py makemigrations 
- python manage.py migrate